필요한 라이브러리 import

In [1]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [2]:
cd "/content/gdrive/MyDrive/2024_MCL_Internship/MNIST"

/content/gdrive/MyDrive/2024_MCL_Internship/MNIST


In [ ]:
import torch
from torch.utils import data
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torch.nn as nn

from matplotlib import pyplot as plt
import os
from time import time

Dataset 정의

In [ ]:
class mnist_train(data.Dataset):
    def __init__(self):
        self.bytes_images = list(open(f"dataset/train-images.idx3-ubyte", "rb").read()[16:])
        self.bytes_labels = list(open("dataset/train-labels.idx1-ubyte", "rb").read()[8:])

    def __getitem__(self, idx):
        image = torch.Tensor(self.bytes_images[idx * 784:(idx + 1) * 784])
        image = torch.reshape(image, (1, 28, 28)) / 255.0
        label = self.bytes_labels[idx]

        return image, label

    def __len__(self):
        return len(self.bytes_labels)

class mnist_test(data.Dataset):
    def __init__(self):
        self.bytes_images = list(open(f"dataset/t10k-images.idx3-ubyte", "rb").read()[16:])
        self.bytes_labels = list(open("dataset/t10k-labels.idx1-ubyte", "rb").read()[8:])

    def __getitem__(self, idx):
        image = torch.Tensor(self.bytes_images[idx * 784:(idx + 1) * 784])
        image = torch.reshape(image, (1, 28, 28)) / 255.0
        label = self.bytes_labels[idx]

        return image, label

    def __len__(self):
        return len(self.bytes_labels)

 데이터 불러오기

In [ ]:
import random

train_dataset = mnist_train()
test_dataset = mnist_test()

print("total length of train dataset : ", len(train_dataset))
print("total length of test dataset : ", len(test_dataset))
i = random.randrange(len(train_dataset))
image_train, label_train = train_dataset[i]
image_train_PIL = transforms.ToPILImage()(image_train)
display(image_train_PIL)
print(label_train)
print()

i = random.randrange(len(test_dataset))
image_test, label_test = test_dataset[i]
image_test_PIL = transforms.ToPILImage()(image_test)
display(image_test_PIL)
print(label_test)

model 정의

In [ ]:
class Linear_Model(nn.Module):
    def __init__(self, is_BN):
        super(Linear_Model, self).__init__()
        self.flatten = nn.Flatten()
        if is_BN:
            self.linear = nn.Sequential(
                nn.Linear(784, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.BatchNorm1d(64),
                nn.ReLU(),
                nn.Linear(64, 10),
                nn.Softmax(dim=1)
            )
        else:
            self.linear = nn.Sequential(
                nn.Linear(784, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10),
                nn.Softmax(dim=1)
            )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear(x)
        return logits

train 코드

In [ ]:
def train_model(info):
    if info['is_BN']:
        isBN = f'withBN'
    else:
        isBN = f'withoutBN'

    print(f"Model : {info['model']}, Batch Normalization : {isBN}")
    epochs = info["epochs"]

    writer = info["writer"]

    ckpt_path = os.path.join('checkpoints', f"{info['model']}_{isBN}")
    if not os.path.exists('checkpoints'): os.mkdir('checkpoints')
    if not os.path.exists(ckpt_path): os.mkdir(ckpt_path)

    train_dataset = mnist_train()
    test_dataset = mnist_test()
    train_dataloader = data.DataLoader(dataset=train_dataset, batch_size=info["batch_size"], shuffle=True,
                                       pin_memory=True)
    test_dataloader = data.DataLoader(dataset=test_dataset, batch_size=info["batch_size"], shuffle=False,
                                      pin_memory=True)

    # 모델 선언, gpu loading
    model = Linear_Model(info["is_BN"])
    model.cuda()

    # Loss Function, Optimizer 선언
    loss_function = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=info["lr"])

    # 에폭별 loss, test acc 담을 list
    log_loss_train = []
    log_loss_test = []
    log_acc_test = []

    # epoch 반복문, iteration, 네트워크 train, epoch별 test
    for epoch in range(epochs):
        # train 준비
        start_time = time()
        train_loss_per_epoch = 0
        model.train()

        # mini batch 불러오기 반복문
        for iter, batch in enumerate(train_dataloader):
            image = batch[0].cuda()
            label = batch[1].cuda()

            y = model(image) # y size : Batch x 10
            loss_train = loss_function(y, label)

            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()

            train_loss_per_epoch += loss_train.item() / len(train_dataloader)

            # tensorboard에 step별 train loss 저장
            writer.add_scalar(f"{info['model']}_{isBN}/Train_Loss", loss_train, epoch * len(train_dataloader) + iter)

        # 에폭별 train 결과 출력해보기
        print(f"Epoch[{epoch}] Loss: {train_loss_per_epoch:.4f}", end="")
        log_loss_train.append(train_loss_per_epoch)

        # test 준비
        model.eval()
        loss_test_epoch = 0
        total_num_accs = 0

        # test 시에 tensor에 gradient 위한 기록 남길 필요 없음
        with torch.no_grad():
            # test에서 mini-batch 불러오기
            for iter, batch in enumerate(test_dataloader):
                image = batch[0].cuda()
                label = batch[1].cuda()

                y = model(image) # size of y : Batch x 10
                loss_test = loss_function(y, label)

                loss_test_epoch += loss_test.item() / len(test_dataloader)

                y_ = torch.argmax(y, dim=1) # size of y_ : batch x 1
                num_accs = torch.sum(y_ == label).item()
                total_num_accs += num_accs / (len(test_dataloader) * info["batch_size"])


        # tensorboard에 test 결과 기록 (Loss, Accuracy)
        writer.add_scalar(f"{info['model']}_{isBN}/Test_Loss", loss_test_epoch, epoch)
        writer.add_scalar(f"{info['model']}_{isBN}/Test_Accuracy", total_num_accs * 100, epoch)
        log_loss_test.append(loss_test_epoch)
        log_acc_test.append(total_num_accs * 100)


        # 네트워크 파라미터 중간 저장
        if (epoch + 1) % 10 == 0:
            torch.save({'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict()}, f'{ckpt_path}/Epoch{epoch:03d}.pth')

        time_per_epoch = time() - start_time
        print(
            f" Test Loss: {loss_test_epoch:.4f} Accuracy Rate: {total_num_accs * 100:.2f}% time per epoch: {time_per_epoch:.2f}sec")

    return log_loss_train, log_loss_test, log_acc_test

Linear 네트워크 main 함수

In [ ]:
if not os.path.exists('logs'): os.mkdir('logs')
writer = SummaryWriter('logs')

info = {
    "epochs" : 100,
    "batch_size" : 500,
    "lr" : 0.01,
    "model" : "Linear_Model",
    "is_BN" : True,
    "writer" : writer
}

linear_BN = train_model(info)

info["is_BN"] = False
linear_withoutBN = train_model(info)

학습 결과를 그래프로 plot

In [ ]:
# --- Logs Visualization
# training loss
plt.plot(linear_BN[0], label='Linear_withBN')
plt.plot(linear_withoutBN[0], label='Linear_withoutBN')
plt.xticks(list(range(0, info["epochs"], info["epochs"] // 5)))
plt.show()
plt.savefig('logs/training_loss_Linear.png')
plt.clf()

# test loss
plt.plot(linear_BN[1], label='Linear_withBN')
plt.plot(linear_withoutBN[1], label='Linear_withoutBN')
plt.xticks(list(range(0, info["epochs"], info["epochs"] // 5)))
plt.show()
plt.savefig('logs/test_loss_Linear.png')
plt.clf()

# test accuracy
plt.plot(linear_BN[2], label='Linear_withBN')
plt.plot(linear_withoutBN[2], label='Linear_withoutBN')
plt.xticks(list(range(0, info["epochs"], info["epochs"] // 5)))
plt.show()
plt.savefig('logs/test_acc_Linear.png')
plt.clf()

tensorboard 실행

In [ ]:
%load_ext tensorboard

%tensorboard --logdir logs

손글씨 직접 써보기 예시 - image pre-processing

In [ ]:
from PIL import Image
import numpy as np
import torchvision.transforms as TF
import cv2

img3 = np.array(Image.open(f'3.png'))
img5 = np.array(Image.open(f'5.png'))
img7 = np.array(Image.open(f'7.png'))
img3 = cv2.resize(img3, (28, 28), interpolation = cv2.INTER_AREA)
img5 = cv2.resize(img5, (28, 28), interpolation = cv2.INTER_AREA)
img7 = cv2.resize(img7, (28, 28), interpolation = cv2.INTER_AREA)
img3 = Image.fromarray(img3).convert('L')
img5 = Image.fromarray(img5).convert('L')
img7 = Image.fromarray(img7).convert('L')
display(img3)
display(img5)
display(img7)

img3 = TF.ToTensor()(img3)
img5 = TF.ToTensor()(img5)
img7 = TF.ToTensor()(img7)

img3 = img3.unsqueeze(0).cuda()
img5 = img5.unsqueeze(0).cuda()
img7 = img7.unsqueeze(0).cuda()

model = Linear_Model(True)
model.cuda()
ckpt = torch.load(f'checkpoints/Linear_Model_withBN/Epoch099.pth')
model.load_state_dict(ckpt['model_state_dict'])
model.eval()
with torch.no_grad():
    logits = model(img3)
    print(torch.argmax(logits).item())
    print()
    logits = model(img5)
    print(torch.argmax(logits).item())
    print()
    logits = model(img7)
    print(torch.argmax(logits).item())